In [ ]:
import logging
import os
import pandas as pd

from pathlib import Path

%load_ext autoreload
%autoreload 2

# Disable logging of the program in the notebook
os.environ["LOGLEVEL"] = "CRITICAL"

logging.basicConfig()
logging.root.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)

logger = logging.getLogger('__main__')
logger.setLevel(os.environ.get("LOGLEVEL", logging.FATAL))

pybooklogger = logging.getLogger('pybook')
pybooklogger.setLevel(logging.DEBUG)

import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.path.abspath("__file__")), '..')))
from setup import *
from tad_help import TadRunner

# Track Layout
Calculate the layout of the dutch railway system


In [ ]:
layout_file =   "../data/prorail/parsed/netherlands-schiphol.json"
start = time.time()
layout = Layout(layout_file)
end = time.time()
print(f"Computed the routing graph for entire Dutch railway network in {end - start} seconds.")
print(f"Number of track parts in the track graph: {len(layout.g.nodes)} with {len(layout.g.edges)} connections.")
print(f"Number of blocks in the routing graph: {len(layout.g_block.nodes)} with {len(layout.g_block.edges)} routes.")

In [ ]:
edges_df = pd.DataFrame({"Outgoing routes": [len(n.outgoing) for n in layout.g_block.nodes.values() if len(n.outgoing) <= 25]})
hist = edges_df.hist(bins=25, )
plt.xlabel("Number of outgoing routes")
plt.ylabel("Number of occurrences")
plt.savefig("results/case_study_eurostar/outgoing_routes.png")
plt.show()

# Experiment Runtime
Take a route of an agent with many stops, and run from start to every stop as an experiment

## Scenario

In [ ]:
scenario_files = Path("../data/case_study")
save_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), "results", "case_study_eurostar")
Path(save_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
os.environ["LOGLEVEL"] = "INFO"

all_data = {"paths": [], "times": [], "stats": []}

for scenario_file in os.listdir(scenario_files):
    scenario_dir = os.path.join(save_dir, scenario_file.split(".")[0])
    Path(scenario_dir).mkdir(parents=True, exist_ok=True)

    logger.info(f"Now working on scenario: {scenario_file}")
    tad_exp = TadRunner(layout, scenario_files / scenario_file, save_dir)
    experiments = tad_exp.run("1800", "e", "RTD|13", "SHL|5", default_direction=1, timeout=1800, 
                              startTime=0, trainTypes=["EUROSTAR"], stop=[])
    experiments[0].metadata = {'color': 'Red',   'label': '@MAEDeR',  'offset': 0, 'linestyle': 3}
    experiments[1].metadata = {'color': 'Blue',  'label': 'FlexSIPP', 'offset': 0, 'linestyle': 3}
    experiments[2].metadata = {'color': 'Green', 'label': 'rSIPP',    'offset': 0, 'linestyle' :3}

    # tad_exp.plot(experiments, save="eurostar", y_offset=2100, y_range=1200, include_expected_arrival=False)
    # tad_exp.plot([experiments[0]], save="eurostar/atsipp", y_offset=3040, include_expected_arrival=False)
    # tad_exp.plot([experiments[1]], save="eurostar/flexsipp", y_offset=1240, include_expected_arrival=False)
    # tad_exp.plot([experiments[2]], save="eurostar/rsipp", y_offset=0, include_expected_arrival=False)

    path_data = get_path_data(experiments, tad_exp.agent_df, scenario=scenario_file)
    time_data = [exp.get_running_time() | exp.get_label() | {'scenario': scenario_file} for exp in experiments]
    comp_data = [exp.get_complexity()   | exp.get_label() | {'scenario': scenario_file} for exp in experiments]
    all_data["paths"].extend(path_data)
    all_data["stats"].extend(comp_data)
    all_data["times"].extend(time_data)

    # Save individual data to tad_exp.save_dir/eurostar.csv
    path_df = pd.DataFrame(path_data)
    path_df.to_csv(scenario_dir  + "/eurostar.csv")

    time_df = pd.DataFrame(time_data)
    time_df.to_csv(scenario_dir + "/eurostar-time.csv")

    comp_df = pd.DataFrame(comp_data)
    comp_df.to_csv(scenario_dir + "/eurostar-comp.csv")


In [ ]:
all_data

In [ ]:
all_path_df = pd.DataFrame(all_data["paths"])
all_path_df.to_csv(os.path.join(os.path.dirname(scenario_dir), "eurostar_all.csv"))

all_stat_df = pd.DataFrame(all_data["stats"])
all_stat_df.to_csv(os.path.join(os.path.dirname(scenario_dir), "eurostar-comp_all.csv"))

all_time_df = pd.DataFrame(all_data["times"])
all_time_df.to_csv(os.path.join(os.path.dirname(scenario_dir), "eurostar-time_all.csv"))